In [2]:
import os
import uuid

DATASET_PATH = 'G:\My Drive\public_data'

In [3]:
import os
import json
import argparse
from pathlib import Path
from tqdm import tqdm

def create_dataset_json(root_dir, output_json, find_description=False):
    """
    Tạo file JSON chứa thông tin về tất cả các mô hình trong dataset

    Args:
        root_dir (str): Đường dẫn đến thư mục gốc của dataset (public-data)
        output_json (str): Đường dẫn để lưu file JSON
        find_description (bool): Tìm file mô tả nếu có (mặc định: False)

    Returns:
        dict: Dữ liệu JSON đã tạo
    """
    # Chuẩn hóa đường dẫn gốc
    # root_dir = os.path.abspath(root_dir)

    # Tạo cấu trúc dữ liệu
    dataset_json = {}

    # Đếm số lượng mô hình để hiển thị tiến trình
    total_models = 0
    for dir_level1 in os.listdir(root_dir):
        level1_path = os.path.join(root_dir, dir_level1)
        if os.path.isdir(level1_path):
            for dir_level2 in os.listdir(level1_path):
                level2_path = os.path.join(level1_path, dir_level2)
                if os.path.isdir(level2_path):
                    if any(file.endswith('.obj') for file in os.listdir(level2_path)):
                        total_models += 1

    print(f"Đã tìm thấy {total_models} thư mục chứa mô hình")

    # Duyệt qua cấu trúc thư mục
    with tqdm(total=total_models, desc="Đang xử lý mô hình") as pbar:
        for dir_level1 in sorted(os.listdir(root_dir)):
            level1_path = os.path.join(root_dir, dir_level1)

            # Chỉ xử lý thư mục
            if not os.path.isdir(level1_path):
                continue

            # Tạo entry cho thư mục cấp 1
            dataset_json[dir_level1] = {}

            for dir_level2 in sorted(os.listdir(level1_path)):
                level2_path = os.path.join(level1_path, dir_level2)

                # Chỉ xử lý thư mục
                if not os.path.isdir(level2_path):
                    continue

                # Kiểm tra xem có file .obj nào không
                obj_files = [f for f in os.listdir(level2_path) if f.endswith('.obj')]
                if not obj_files:
                    continue

                # Tạo entry cho thư mục cấp 2
                object_data = {}

                # Tìm các file theo thứ tự ưu tiên
                # 1. normalized_model.obj (nếu có)
                # 2. model.obj (nếu có)
                # 3. Bất kỳ file .obj nào khác
                obj_path = None
                if "normalized_model.obj" in obj_files:
                    obj_path = os.path.join(level2_path, "normalized_model.obj")
                if "model.obj" in obj_files:
                    obj_path = os.path.join(level2_path, "model.obj")
                else:
                    obj_path = os.path.join(level2_path, obj_files[0])

                # Chuyển đổi đường dẫn tuyệt đối thành đường dẫn tương đối so với root_dir
                rel_obj_path = os.path.relpath(obj_path, root_dir)
                object_data["obj_path"] = rel_obj_path

                # Tìm file texture.png
                texture_path = os.path.join(level2_path, "texture.png")
                if os.path.exists(texture_path):
                    rel_texture_path = os.path.relpath(texture_path, root_dir)
                    object_data["texture_path"] = rel_texture_path

                # Tìm file image.png hoặc bất kỳ file ảnh nào khác
                image_files = [f for f in os.listdir(level2_path) if f.endswith(('.png', '.jpg', '.jpeg')) and f != "texture.png"]
                if image_files:
                    image_path = os.path.join(level2_path, image_files[0])
                    rel_image_path = os.path.relpath(image_path, root_dir)
                    object_data["image_path"] = rel_image_path

                # Tìm file mô tả nếu được yêu cầu
                if find_description:
                    description_files = [f for f in os.listdir(level2_path) if f.endswith(('.txt', '.md')) and "description" in f.lower()]
                    if description_files:
                        desc_path = os.path.join(level2_path, description_files[0])
                        try:
                            with open(desc_path, 'r', encoding='utf-8') as f:
                                description = f.read().strip()
                            object_data["description"] = description
                        except Exception as e:
                            print(f"Lỗi khi đọc file mô tả {desc_path}: {e}")
                    else:
                        object_data["description"] = ""

                # Thêm thông tin mô hình vào dataset
                dataset_json[dir_level1][dir_level2] = object_data

                # Cập nhật thanh tiến trình
                pbar.update(1)

    # Lưu dữ liệu vào file JSON
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(dataset_json, f, ensure_ascii=False, indent=2)

    print(f"Đã lưu thông tin dataset vào file: {output_json}")

    # Thống kê
    total_models = sum(len(dirs) for dirs in dataset_json.values())
    print(f"Tổng số thư mục cấp 1: {len(dataset_json)}")
    print(f"Tổng số mô hình: {total_models}")

    return dataset_json

def parse_args():
    parser = argparse.ArgumentParser(description='Tạo file JSON cho dataset')
    parser.add_argument('--root_dir', type=str, required=True,
                        help='Đường dẫn đến thư mục gốc của dataset (public-data)')
    parser.add_argument('--output_json', type=str, default='dataset.json',
                        help='Đường dẫn để lưu file JSON (mặc định: dataset.json)')
    parser.add_argument('--find_description', action='store_true',
                        help='Tìm file mô tả nếu có')
    return parser.parse_args()

def main():
    # args = parse_args()
    create_dataset_json(DATASET_PATH, "object.json", False)

if __name__ == "__main__":
    main()

Đã tìm thấy 34051 thư mục chứa mô hình


Đang xử lý mô hình: 100%|██████████| 34051/34051 [07:23<00:00, 76.71it/s] 


Đã lưu thông tin dataset vào file: object.json
Tổng số thư mục cấp 1: 1622
Tổng số mô hình: 34051


In [1]:
import json

In [2]:
with open("G:/My Drive/public_data/object.json", 'r', encoding='utf-8') as f:
    dataset_json = json.load(f)

uuid1s = [key for key in dataset_json.keys()]